In [2]:
import os
import re

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
RAW_DATA_PATH = "../data/raw/"
PROCESSED_DATA_PATH = "../data/processed/"

## Очистка данных

### Данные пациентов (1 табл)

In [3]:
patients_df = pd.read_csv(os.path.join(RAW_DATA_PATH, "данные_пациентов.csv"))
patients_df.shape

(413799, 5)

In [4]:
patients_df = patients_df.drop_duplicates(keep="first")
patients_df.shape

(379246, 5)

Пациенты могли переезжать. Предполагаем, что id пациента отражают время регистрации => данные отранжированы по времени. И оставляем только последние записи для каждого пациента в каждом районе проживания.

In [5]:
patients_df = patients_df.sort_values("id_пациента").reset_index(drop=True)
patients_df = patients_df.drop_duplicates(subset=["id_пациента"], keep="last")
patients_df.shape

(379076, 5)

In [6]:
patients_df["дата_рождения"] = pd.to_datetime(
    patients_df["дата_рождения"],
    format="%d.%m.%Y",
    dayfirst=True,
    errors="coerce",
)

In [7]:
patients_df["пол"] = patients_df["пол"].str.lower()
patients_df["район_проживания"] = patients_df["район_проживания"].str.lower()
patients_df["регион"] = patients_df["регион"].str.lower()

In [16]:
patients_df.rename(columns={"id_пациента": "id"}, inplace=True)

In [18]:
patients_df.to_csv("../data/processed/patients.csv", index=False)

### Данные рецептов

In [3]:
recipes_df = pd.read_csv("../data/raw/данные_рецептов.csv")
recipes_df.head()

,id_пациента,дата_рецепта,код_диагноза,код_препарата,id_пациента.1
0,24419779,2022-03-31 11:52:30,F06,200900018,1226228
1,17890856,2019-06-04 18:41:10,I11.9,201300167,62857
2,18749669,2019-10-29 12:35:31,K86.1,201300027,1186134
3,31412579,2024-07-30 17:06:59,I67.8,202200024,1927759
4,21191943,2020-12-01 19:59:02,E10.9,201200267,1265025


In [4]:
recipes_df.isna().sum()

id_пациента         0
дата_рецепта     1178
код_диагноза        0
код_препарата       0
id_пациента.1       0
dtype: int64

In [5]:
recipes_df.dropna(inplace=True)

`id_пациента` не нужен

In [6]:
recipes_df = recipes_df.drop(columns=["id_пациента"])
recipes_df = recipes_df.rename(columns={"id_пациента.1": "id_пациента"})

ранжируем по дате выписки рецепта

In [7]:
recipes_df["дата_рецепта"] = pd.to_datetime(recipes_df["дата_рецепта"])
recipes_df = recipes_df.sort_values("дата_рецепта")

In [8]:
recipes_df["код_диагноза"] = recipes_df["код_диагноза"].str.lower()

удаляем дубликаты

In [9]:
cleaned_recipes = recipes_df.drop_duplicates(keep="first")

In [10]:
cleaned_recipes.to_csv("../data/processed/recipes.csv", index=False)

### Данные препаратов

In [21]:
medication_df = pd.read_csv("../data/raw/данные_препараты.csv")

In [22]:
medication_df.duplicated(subset=["код_препарата"]).sum()

np.int64(0)

In [23]:
medication_df.isna().sum()

код_препарата           0
дозировка             116
Торговое название       0
стоимость            2201
Полное_название         0
dtype: int64

In [24]:
medication_df.columns = [
    "код_препарата",
    "дозировка",
    "торговое_название",
    "стоимость",
    "мета_информация",
]

In [30]:
medication_df["дозировка"] = medication_df["дозировка"].fillna("unknown").str.lower()
medication_df["стоимость"] = medication_df["стоимость"].fillna(-1).astype(int)

In [47]:
medication[""] = medication["price"].fillna(0)

In [49]:
medication.to_csv("../data/processed/medication.csv", index=False)

### Данные диагнозов

In [50]:
diagnoses = pd.read_csv("../data/raw/данные_диагнозы.csv")

print("=" * 80)
print("HEAD".center(80))
print("=" * 80)
print(diagnoses.head())

print("\n" + "=" * 80)
print("INFO".center(80))
print("=" * 80)
print(diagnoses.info())

                                      HEAD                                      
  код_мкб                                  название_диагноза  \
0     A00                                             Холера   
1   A00.0  Холера, вызванная холерным вибрионом 01, биова...   
2   A00.1  Холера, вызванная холерным вибрионом 01, биова...   
3   A00.9                                Холера неуточненная   
4     A01                                      Тиф и паратиф   

                               класс_заболевания  
0  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
1  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
2  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
3  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  
4  НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ  

                                      INFO                                      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14801 entries, 0 to 14800
Data columns (total 3 columns):
 #   Column             Non-Null Count

In [51]:
diagnoses.columns = ["diagnosis_code", "name", "classification"]

In [53]:
diagnoses["diagnosis_code"] = diagnoses["diagnosis_code"].str.upper()

In [55]:
diagnoses.to_csv("../data/processed/diagnoses.csv", index=False)

## Инсайты